<a href="https://colab.research.google.com/github/swarnava-96/Pyspark/blob/main/Pyspark6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Pyspark ML

In [3]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 67kB/s 
     |████████████████████████████████| 204kB 18.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=291c16157bafa17111340b954fe7d2b7b7f5011bdb39e454910dcaabcec6dfd4
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [5]:
spark

In [6]:
## Read The dataset
training = spark.read.csv('/content/drive/MyDrive/Datasets/test1.csv',header=True,inferSchema=True)

In [7]:
training.show()

+--------+---+----------+------+
|    Name|Age|Experience|Salary|
+--------+---+----------+------+
|Swarnava| 25|         0|     0|
|Abhishek| 24|         2| 30000|
| Pushpok| 24|         6| 50000|
|  Deeyan| 24|         2| 35000|
| Swastik| 24|         1| 40000|
|  Pranoy| 23|         4| 65000|
+--------+---+----------+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [10]:
from pyspark.ml.feature import VectorAssembler
featureAssembler=VectorAssembler(inputCols=['Age','Experience'],outputCol='Independent Features')

In [11]:
output=featureAssembler.transform(training)

In [12]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|Age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|Swarnava| 25|         0|     0|          [25.0,0.0]|
|Abhishek| 24|         2| 30000|          [24.0,2.0]|
| Pushpok| 24|         6| 50000|          [24.0,6.0]|
|  Deeyan| 24|         2| 35000|          [24.0,2.0]|
| Swastik| 24|         1| 40000|          [24.0,1.0]|
|  Pranoy| 23|         4| 65000|          [23.0,4.0]|
+--------+---+----------+------+--------------------+



In [13]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [16]:
finalized_data=output.select(['Independent Features','Salary'])

In [18]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [25.0,0.0]|     0|
|          [24.0,2.0]| 30000|
|          [24.0,6.0]| 50000|
|          [24.0,2.0]| 35000|
|          [24.0,1.0]| 40000|
|          [23.0,4.0]| 65000|
+--------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression


##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor=regressor.fit(train_data)


In [20]:
### Coefficients

regressor.coefficients

DenseVector([-23750.0, 4375.0])

In [21]:
### Intercepts

regressor.intercept

593750.000000153

In [22]:
### Prediction

pred_results=regressor.evaluate(test_data)

In [23]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [24.0,1.0]| 40000|28125.00000000291|
|          [24.0,2.0]| 30000|32500.00000000163|
|          [24.0,2.0]| 35000|32500.00000000163|
+--------------------+------+-----------------+



In [24]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(5624.99999999903, 51171874.99997697)